In [ ]:
#|default_exp rag

# RAG Support for ShellSage

Implementing RAG functionality using local man pages.

In [ ]:
#|export
import lancedb
from chonkie import RecursiveChunker
from model2vec import StaticModel
import subprocess
import re
import logging
import os
import pyarrow as pa
import numpy as np
import pandas as pd

logging.basicConfig(level=logging.INFO)

In [ ]:
#|export
def init_db(db_path="man_index.lance"):
    """Initialize or open a LanceDB database."""
    os.makedirs(os.path.dirname(db_path) if os.path.dirname(db_path) else '.', exist_ok=True)
    return lancedb.connect(db_path)

def create_chunks_table(db):
    """Create or replace the man page chunks table."""
    data = pd.DataFrame({
        'title': [''],
        'section': [''],
        'chunk': [''],
        'vector': [[0.0] * 256]
    })
    return db.create_table("man_chunks", data=data, mode="overwrite")

In [ ]:
#|export
def get_man_pages():
    """Get all available man pages on the system."""
    result = subprocess.run(['apropos', '-l', '.'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    pages = []
    
    for line in result.stdout.splitlines():
        if not line.strip(): continue
        try:
            name, section = line.split('(', 1)
            section = section.split(')', 1)[0]
            name = name.strip()
            
            path = subprocess.check_output(['man', '-w', name], text=True).strip()
            if os.path.exists(path):
                pages.append({
                    'title': name,
                    'section': section,
                    'path': path
                })
        except Exception as e:
            logging.debug(f"Skipping line '{line}': {str(e)}")
            continue
            
    return pages

In [ ]:
#|export
def read_man_page(path):
    """Read a man page and return its text content."""
    try:
        result = subprocess.run(['man', path], capture_output=True, text=True)
        return result.stdout if result.returncode == 0 else ""
    except Exception as e:
        logging.warning(f"Failed to read man page {path}: {str(e)}")
        return ""

In [ ]:
#|export
def chunk_text(text):
    """Chunk text using Chonkie's RecursiveChunker."""
    chunker = RecursiveChunker(
        tokenizer="gpt2",
        chunk_size=512,
        min_characters_per_chunk=12
    )
    chunks = chunker.chunk(text)
    return [chunk.text for chunk in chunks]

In [ ]:
#|export
def get_embeddings(texts):
    """Get embeddings using Model2Vec."""
    model = StaticModel.from_pretrained("minishlab/M2V_base_output")
    vectors = model.encode(texts)
    return [vector.astype(np.float32).tolist() for vector in vectors]

In [ ]:
#|export
def index_man_pages(db_path="man_index.lance"):
    """Index all man pages into the vector database."""
    db = init_db(db_path)
    table = create_chunks_table(db)
    
    pages = get_man_pages()
    logging.info(f"Found {len(pages)} man pages to index")
    
    all_data = []
    total_chunks = 0
    
    for page in pages:
        try:
            logging.info(f"Processing {page['title']}({page['section']})")
            text = read_man_page(page['path'])
            if not text:
                logging.warning(f"Empty content for {page['title']}({page['section']})")
                continue
                
            chunks = chunk_text(text)
            if not chunks:
                logging.warning(f"No chunks created for {page['title']}({page['section']})")
                continue
                
            vectors = get_embeddings(chunks)
            
            for chunk, vector in zip(chunks, vectors):
                all_data.append({
                    "title": page['title'],
                    "section": page['section'],
                    "chunk": chunk,
                    "vector": vector
                })
            
            total_chunks += len(chunks)
            logging.info(f"Processed {len(chunks)} chunks for {page['title']} (total: {total_chunks})")
                
        except Exception as e:
            logging.error(f"Failed to process {page['title']}({page['section']}): {str(e)}")
            continue
    
    if all_data:
        df = pd.DataFrame(all_data)
        table.add(df)
        logging.info(f"Added all {len(df)} chunks to database")
    
    return total_chunks

In [ ]:
#|export
def query_man_pages(query: str, top_k: int = 5, db_path: str = "man_index.lance"):
    """Query the indexed man pages and return the most relevant chunks."""
    db = init_db(db_path)
    table = db.open_table("man_chunks")
    
    # Get query embedding
    query_vector = get_embeddings([query])[0]
    
    # Search the database
    results = table.search(query_vector).limit(top_k).to_pandas()
    
    return [{
        'title': row['title'],
        'section': row['section'],
        'chunk': row['chunk'],
        'score': row['_distance']
    } for _, row in results.iterrows()]

In [ ]:
# Test the implementation
test_db = "test_man_index.lance"
if os.path.exists(test_db):
    import shutil
    shutil.rmtree(test_db)

print("Starting man page indexing...")
total_chunks = index_man_pages(test_db)

# Verify database contents
db = lancedb.connect(test_db)
table = db.open_table("man_chunks")
df = table.to_pandas()
print(f"\nDatabase statistics:")
print(f"Total chunks indexed: {len(df)}")
print(f"Expected chunks: {total_chunks}")

# Test querying
print("\nTesting query functionality...")
query = "how to create a jar file"
results = query_man_pages(query, top_k=3, db_path=test_db)

print(f"\nTop 3 results for query: '{query}'")
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result['title']}({result['section']}) - Score: {result['score']:.4f}")
    print(f"Chunk: {result['chunk'][:200]}...")